In [1]:
import nltk; nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /Users/Owner/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
!pip install --upgrade gensim
!pip install -U spacy
!pip install -U pip setuptools wheel
!pip install pyldavis

In [3]:
!python3 -m spacy download en_core_web_sm

     |████████████████████████████████| 13.9 MB 9.5 MB/s            
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [4]:
!pip install pyLDAvis==3.2.1

In [5]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

import gensim
import gensim.models
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

import spacy

import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [6]:
# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

In [7]:
all = pd.read_csv('Customer Reviews Data.csv')

In [8]:
#tokenize words and clean text
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(all['Statement']))

print(data_words[:1])

[['the', 'assistance', 'service', 'is', 'completely', 'useless', 'problem', 'reported', 'over', 'months', 'ago', 'by', 'email', 'and', 'phone', 'and', 'they', 'always', 'reply', 'sending', 'the', 'same', 'stupid', 'template', 'of', 'escalation', 'to', 'the', 'relevant', 'department', 'shame', 'on', 'you']]


In [9]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

['the', 'assistance', 'service', 'is', 'completely', 'useless', 'problem', 'reported', 'over', 'months', 'ago', 'by', 'email', 'and', 'phone', 'and', 'they', 'always', 'reply', 'sending', 'the', 'same', 'stupid', 'template', 'of', 'escalation', 'to', 'the', 'relevant', 'department', 'shame', 'on', 'you']


In [10]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [11]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

[['assistance', 'service', 'completely', 'useless', 'problem', 'report', 'month', 'ago', 'email', 'phone', 'always', 'reply', 'send', 'stupid', 'template', 'escalation', 'relevant', 'department', 'shame']]


In [12]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1)]]


In [13]:
id2word[0]

'ago'

In [14]:
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('ago', 1),
  ('always', 1),
  ('assistance', 1),
  ('completely', 1),
  ('department', 1),
  ('email', 1),
  ('escalation', 1),
  ('month', 1),
  ('phone', 1),
  ('problem', 1),
  ('relevant', 1),
  ('reply', 1),
  ('report', 1),
  ('send', 1),
  ('service', 1),
  ('shame', 1),
  ('stupid', 1),
  ('template', 1),
  ('useless', 1)]]

In [15]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [16]:
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.055*"useless" + 0.048*"good" + 0.039*"need" + 0.036*"bill" + 0.035*"stop" '
  '+ 0.035*"let" + 0.034*"even" + 0.026*"see" + 0.024*"disgrace" + '
  '0.024*"total"'),
 (1,
  '0.045*"expect" + 0.027*"prosit" + 0.027*"reply" + 0.027*"query" + '
  '0.016*"arm" + 0.014*"capable" + 0.014*"deserve" + 0.014*"mizzi" + '
  '0.014*"people" + 0.014*"support"'),
 (2,
  '0.047*"service" + 0.040*"work" + 0.040*"take" + 0.040*"customer" + '
  '0.037*"time" + 0.033*"care" + 0.033*"system" + 0.031*"arm" + 0.028*"joke" + '
  '0.028*"hour"'),
 (3,
  '0.083*"thing" + 0.049*"make" + 0.016*"time" + 0.016*"simple" + '
  '0.016*"waste" + 0.016*"open" + 0.016*"hour" + 0.016*"excellent" + '
  '0.001*"service" + 0.001*"sign"'),
 (4,
  '0.072*"change" + 0.046*"bother" + 0.027*"option" + 0.027*"buy" + '
  '0.027*"star" + 0.025*"name" + 0.021*"last" + 0.017*"service" + '
  '0.017*"ownership" + 0.017*"reply"'),
 (5,
  '0.090*"help" + 0.080*"bill" + 0.042*"send" + 0.033*"month" + 0.026*"refund" '
  '+ 0.024*"

In [17]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -8.813577213746504

Coherence Score:  0.35705021624518285


In [18]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, dictionary=lda_model.id2word, mds='mmds')   
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
5     -0.228105 -0.058203       1        1  11.550139
7     -0.328270 -0.043878       2        1   9.860759
14    -0.110050 -0.296742       3        1   8.472546
0      0.092647 -0.273902       4        1   7.835289
2      0.177997  0.248202       5        1   7.248401
19    -0.221171  0.132281       6        1   6.531905
6     -0.118000  0.259317       7        1   6.043153
13    -0.195562 -0.194384       8        1   5.891950
16     0.273941 -0.083159       9        1   5.398433
15     0.139964 -0.147056      10        1   4.129100
8      0.040745 -0.152118      11        1   3.863736
9      0.247995  0.085486      12        1   3.817011
10    -0.074701  0.141079      13        1   3.640387
11    -0.032950 -0.087130      14        1   3.452523
4      0.001580  0.199231      15        1   3.450478
12     0.095724  0.154584      16        1   3.223719
18     0.140510  0.022735      17        1   2.080193
17     0.067679  0.023694      18        1   1.433327
1      0.004212  0.058751      19        1   1.263559
3      0.025813  0.011212      20        1   0.813393, topic_info=           Term       Freq      Total Category  logprob  loglift
326        time  27.000000  27.000000  Default   30.000  30.0000
48         bill  80.000000  80.000000  Default   29.000  29.0000
20         help  22.000000  22.000000  Default   28.000  28.0000
92          pay  37.000000  37.000000  Default   27.000  27.0000
14      service  20.000000  20.000000  Default   26.000  26.0000
..          ...        ...        ...      ...      ...      ...
5         email   0.016903   7.949107  Topic20   -6.815  -1.3416
6    escalation   0.016903   1.508524  Topic20   -6.815   0.3203
7         month   0.016903  17.469552  Topic20   -6.815  -2.1290
8         phone   0.016903   4.149224  Topic20   -6.815  -0.6915
9       problem   0.016903   9.204287  Topic20   -6.815  -1.4882

[853 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
328       7  0.310867        able
328      13  0.621734        able
28       16  0.774416  absolutely
410       8  0.760696      access
41        1  0.231285     account
...     ...       ...         ...
136       8  0.393595        year
136      10  0.295197        year
112       1  0.326635         yet
112      10  0.326635         yet
112      14  0.326635         yet

[776 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[6, 8, 15, 1, 3, 20, 7, 14, 17, 16, 9, 10, 11, 12, 5, 13, 19, 18, 2, 4])

In [19]:
pyLDAvis.save_html(vis, 'Customer Reviews.html')